In [2]:
# Bring in all dependencies

import pandas as pd
import requests
import numpy as np
import json
import time
from datetime import date
from pprint import pprint
import sqlalchemy
#from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from sqlalchemy import Column, Integer, String, Float
from config_poly import api_key
import sqlite3
#from splinter import Browser
#from bs4 import BeautifulSoup
#from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# Establish link to Russell 100 index wiki page
russell_url = "https://en.wikipedia.org/wiki/Russell_1000_Index"

# Creaters browser window for scraping
#executable_path = {'executable_path': ChromeDriverManager().install()}
#browser = Browser('chrome', **executable_path, headless=False)

In [4]:
# Visits url, creates soup object
#browser.visit(russell_url)
#soup = BeautifulSoup(browser.html, 'html.parser')

In [5]:
thing = pd.read_html(russell_url)
company_table = thing[2]
companies = company_table["Company"]
myCompanies = pd.unique(companies)
companies

0                       2U
1                       3M
2      Abbott Laboratories
3              AbbVie Inc.
4                  Abiomed
              ...         
952                 Xilinx
953          XPO Logistics
954             Xylem Inc.
955     Yum China Holdings
956            Yum! Brands
Name: Company, Length: 957, dtype: object

In [6]:
tickers = company_table["Ticker"]
myTickers = tickers.drop_duplicates()
len(myTickers)

957

In [7]:
# n = 100
# newTickers = [tickers[i:i + n] for i in range(0, len(tickers), n)]
# Should be 10 lists
# newTickers

In [8]:
# Initialize list to hold information from polygon.io
polygon_info = []
polygon_div = []

startTime = time.time()
#today = date.today()
#todays_date = today.strftime("%Y-%m-%d")

# Initializes counter to 0
count = 0

error_log = open("errors.log","a")
    
# Loops through one list of ticker symbols
for ticker in tickers:

    # Checks if count equals 4, used to prevent exceeding 
    # the API call limit of 5 calls/minute
    if count == 4:

        # Tells program to wait 1.5 minutes before running again
        time.sleep(60)

        # Ticker symbols to be used for API Call
        dataset_code = ticker

        # URLs for API Calls, the first for open and closed prices, 
        # the second for divident information
        url = f"https://api.polygon.io/v1/open-close/{dataset_code}/2021-04-23?unadjusted=true&apiKey={api_key}"
        div_url = f"https://api.polygon.io/v2/reference/dividends/{dataset_code}?&apiKey={api_key}"

        # Saving responses from both urls
        try:
            response = requests.get(url).json()
            div_response = requests.get(div_url).json()

            # Adding responses to info and div lists, respectively
            polygon_info.append(response)
            polygon_div.append(div_response)

        except:
            error_log.write(ticker)
            print(f"On {ticker}, error trying to call")

        # Resets counter
        count = 0

    else:
        # Ticker symbols to be used for API Call
        dataset_code = ticker

        # URLs for API Calls, the first for open and closed prices, 
        # the second for divident information
        url = f"https://api.polygon.io/v1/open-close/{dataset_code}/2021-04-23?unadjusted=true&apiKey={api_key}"
        div_url = f"https://api.polygon.io/v2/reference/dividends/{dataset_code}?&apiKey={api_key}"

         # Saving responses from both urls
        try:
            response = requests.get(url).json()
            div_response = requests.get(div_url).json()

            # Adding responses to info and div lists, respectively
            polygon_info.append(response)
            polygon_div.append(div_response)

        except:
            error_log.write(ticker)
            print(f"On {ticker}, error trying to call")

        # Increase counter by 2
        count += 2



On BKR, error trying to call


In [11]:
executionTime = (time.time() - startTime)

realTime = executionTime/60
realTime

496.2675666451454

In [16]:
error2_log = open("errors_2.log","a")

read_error = open("errors.log","r")
startTime = time.time()

# Loops through one list of ticker symbols
for ticker in read_error:

    # Checks if count equals 4, used to prevent exceeding 
    # the API call limit of 5 calls/minute
    if count == 4:

        # Tells program to wait 1.5 minutes before running again
        time.sleep(60)

        # Ticker symbols to be used for API Call
        dataset_code = ticker

        # URLs for API Calls, the first for open and closed prices, 
        # the second for divident information
        url = f"https://api.polygon.io/v1/open-close/{dataset_code}/2021-04-23?unadjusted=true&apiKey={api_key}"
        div_url = f"https://api.polygon.io/v2/reference/dividends/{dataset_code}?&apiKey={api_key}"

        # Saving responses from both urls
        try:
            response = requests.get(url).json()
            div_response = requests.get(div_url).json()

            # Adding responses to info and div lists, respectively
            polygon_info.append(response)
            polygon_div.append(div_response)

        except:
            error_log.write(ticker)
            print(f"On {ticker}, error trying to call")

        # Resets counter
        count = 0

    else:
        # Ticker symbols to be used for API Call
        dataset_code = ticker

        # URLs for API Calls, the first for open and closed prices, 
        # the second for divident information
        url = f"https://api.polygon.io/v1/open-close/{dataset_code}/2021-04-23?unadjusted=true&apiKey={api_key}"
        div_url = f"https://api.polygon.io/v2/reference/dividends/{dataset_code}?&apiKey={api_key}"

         # Saving responses from both urls
        try:
            response = requests.get(url).json()
            div_response = requests.get(div_url).json()

            # Adding responses to info and div lists, respectively
            polygon_info.append(response)
            polygon_div.append(div_response)

        except:
            error2_log.write(ticker)
            print(f"On {ticker}, error trying to call")

        # Increase counter by 2
        count += 2

In [17]:
executionTime = (time.time() - startTime)

realTime = executionTime/60
realTime

0.047592198848724364

In [18]:
company_table = pd.DataFrame({'Name':myCompanies,"Symbol":myTickers})

In [19]:
polygon_info_df = pd.DataFrame(polygon_info)
del polygon_info_df['status']
del polygon_info_df['afterHours']
del polygon_info_df['preMarket']
polygon_info_df = polygon_info_df[['symbol','from','open','high','low','close','volume']]
polygon_info_df.columns = ['symbol','date','open','high','low','close','volume']

polygon_info_df

,symbol,date,open,high,low,close,volume
0,TWOU,2021-04-23,38.92,41.9500,38.510,40.56,1339545.0
1,MMM,2021-04-23,201.00,203.1550,200.990,202.20,1897775.0
2,ABT,2021-04-23,123.62,124.1100,122.645,123.31,5272865.0
3,ABBV,2021-04-23,109.71,111.7699,109.550,111.38,4671715.0
4,ABMD,2021-04-23,345.59,355.9400,344.790,351.03,291367.0
...,...,...,...,...,...,...,...
951,XLNX,2021-04-23,124.58,130.1800,124.450,129.66,1779902.0
952,XPO,2021-04-23,135.92,138.1250,135.670,136.87,942439.0
953,XYL,2021-04-23,109.65,110.8800,109.250,110.48,479333.0
954,YUMC,2021-04-23,59.02,59.3100,58.520,59.13,1548055.0


In [20]:
all_div = polygon_div[1:]
div_amt_list = []
payDate = []
ticker_list = []

for div in all_div:
    result = div['results']
    
    if result == []:
        continue
        
    else:
        ticker = div["results"][0]['ticker']
        div_amt = div["results"][0]['amount']
        pay_date = div["results"][0]['paymentDate']
    
    ticker_list.append(ticker)
    div_amt_list.append(div_amt)
    payDate.append(pay_date)
    
polygon_div_df = pd.DataFrame({"Symbol":ticker_list,"Amount ($)":div_amt_list,"Payment Date":payDate})
polygon_div_df

,Symbol,Amount ($),Payment Date
0,MMM,1.48,2021-03-12
1,ABT,0.45,2021-02-16
2,ABBV,1.30,2021-02-16
3,ACN,0.88,2021-02-12
4,ATVI,0.41,2020-05-06
...,...,...,...
729,XRX,0.25,2021-04-30
730,XLNX,0.38,2020-12-02
731,XYL,0.28,2021-03-18
732,YUMC,0.12,2021-03-25


In [21]:
polygon_info_df.to_csv('openClose.csv')
polygon_div_df.to_csv('dividend.csv')
company_table.to_csv('companies.csv')

In [22]:
polygon_info_df = pd.read_csv('openClose.csv')
polygon_div_df = pd.read_csv('dividend.csv')
company_table = pd.read_csv('companies.csv')

polygon_info_df.columns = ['index','symbol','date','opening','high','low','close','volume']
polygon_info_df = polygon_info_df.drop(columns='index')
polygon_div_df

,Unnamed: 0,Symbol,Amount ($),Payment Date
0,0,MMM,1.48,2021-03-12
1,1,ABT,0.45,2021-02-16
2,2,ABBV,1.30,2021-02-16
3,3,ACN,0.88,2021-02-12
4,4,ATVI,0.41,2020-05-06
...,...,...,...,...
729,729,XRX,0.25,2021-04-30
730,730,XLNX,0.38,2020-12-02
731,731,XYL,0.28,2021-03-18
732,732,YUMC,0.12,2021-03-25


In [23]:
del polygon_div_df['Unnamed: 0']

In [24]:
del company_table['Unnamed: 0']
company_table

,Name,Symbol
0,2U,TWOU
1,3M,MMM
2,Abbott Laboratories,ABT
3,AbbVie Inc.,ABBV
4,Abiomed,ABMD
...,...,...
952,Xilinx,XLNX
953,XPO Logistics,XPO
954,Xylem Inc.,XYL
955,Yum China Holdings,YUMC


In [25]:
# Create sqlite database path
database_path = "russell_stock.sqlite"

# Create an engine that can talk to the database
engine = create_engine(f"sqlite:///{database_path}")

In [26]:
# use engine to connect to existing tables/db
Base = declarative_base()
#conn = sqlite3.connect('russell_stock.sqlite')
#c = conn.cursor()

# Use this to clear out the db
# ----------------------------------
Base.metadata.drop_all(bind=engine)

# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
#Base.metadata.create_all(engine)

In [27]:
# Define our General_Info table
class Open_Close(Base):
    __tablename__ = 'openClose'
    __table_args__ = {'extend_existing': True}
    symbol = Column(String, primary_key=True)
    date = Column(String)
    opening = Column(Float)
    high = Column(Float)
    low = Column(Float)
    close = Column(Float)
    volume = Column(Integer)

# Define our Dividend table
class Dividend(Base):
    __tablename__ = 'dividend'
    __table_args__ = {'extend_existing': True}
    symbol = Column(String, primary_key=True)
    amount = Column(Float)
    payment_date = Column(String)

# Define our Company Table
class Company(Base):
    __tablename__ = 'company'
    __table_args__ = {'extend_existing': True}
    symbol = Column(String, primary_key=True)
    name = Column(String)

In [28]:
# Save references to each table (capital because they are considered classes) 
#Company = Database.classes.company
#Dividend = Database.classes.dividend
#Open_Close = Database.classes.open_close

In [29]:
#for i in polygon_info_df.index:
#    print(polygon_info_df['from'][i])


# insert dataframe into sqlite tables
company_table.to_sql(name='company', con=engine, if_exists='replace', index=True)
polygon_div_df.to_sql(name='dividend', con=engine, if_exists='replace', index=True)
polygon_info_df.to_sql(name='openClose', con=engine, if_exists='replace', index=True)

In [30]:
# # for i in polygon_info_df.index:
#     thisFrom = polygon_info_df["date"][i]
#     thisSymbol = polygon_info_df["symbol"][i]
#     thisOpen = polygon_info_df["open"][i]
#     thisHigh = polygon_info_df["high"][i]
#    thisLow = polygon_info_df["low"][i]
#    thisClose = polygon_info_df["close"][i]
#    thisVol = polygon_info_df["volume"][i]
    
    #row = [thisFrom,thisSymbol,thisOpen,thisHigh,thisLow,thisClose,thisVol]
    
#    thisCompany = Open_Close(date=thisFrom,symbol=thisSymbol,opening=thisOpen,high=thisHigh,low=thisLow,close=thisClose,volume=thisVol)
#    session = Session(bind=engine)
    #conn.execute('Insert into open_close values(?,?,?,?,?,?,?)', row)
    #engine.execute()
#    Session.configure(bind=engine)
#    session.add(thisCompany)
    #session.commit()
    
#session.close()
    
    
    
    

In [31]:
#c.close()
#session.commit()



In [32]:
#session.close()

#conn.close()

engine.dispose()

In [33]:
engine = create_engine(f"sqlite:///{database_path}")

connection = engine.connect()

# use engine to connect to existing tables/db
base = automap_base()
base.prepare(engine, reflect=True)

# View all of the classes/tables that automap found
thing = base.classes.keys()

print(thing)

# Save references to each table (capital because they are considered classes) 
#Company = Database.classes.company
#Dividend = Database.classes.dividend
inspector = inspect(engine)

[]


In [ ]:
connection.close()

engine.dispose()